In [41]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from pathlib import Path
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer

In [37]:
# Run related files
%run 1_functions.ipynb

In [46]:
# Load Data

# Use absolute path to ensure we find the file
file_path = '/Users/mythrikrishnamurthy/Documents/Python/Popmenu/Lead-Fit-Score-ML-Model/notebook/data/BrizoData.csv'

# Verify file exists before reading
if not os.path.exists(file_path):
    raise FileNotFoundError(f"File not found at: {file_path}\nCurrent directory: {os.getcwd()}")

# Load the data
brizo_data = pd.read_csv(file_path, encoding="latin-1")

/var/folders/_v/vvwjgp3n06l5fx24q03s8pn40000gp/T/ipykernel_5648/109913776.py:11: DtypeWarning: Columns (111,114) have mixed types. Specify dtype option on import or set low_memory=False.
  brizo_data = pd.read_csv(file_path, encoding="latin-1")


In [ ]:
# Replace spaces with underscores in all column names
brizo_data.columns = brizo_data.columns.str.replace(' ', '_')

# Data Overview
brizo_data.shape

(9216, 121)

In [ ]:
# Data Overview
brizo_data.head(5)

,Brizo_ID,Location_ID,Operational_Status,Name_(Establishment),Alt_Name,First_Seen,Phone_(Establishment),URL,Established,Business_Type,...,Technologies_Admin,Technologies_Cx,Technologies_Supply,Technologies_Virtual,Technologies_Reservation,Technologies_Search,Brizo_Foodmetrics_Link,Popmenu_Location_Segment,Restaurant_ID,Retention_Propensity
0,ef5cd4a86eec2763,22495,open,Firefly,NaN,3/1/20,+1(850)249-3359,https://www.fireflypcb.com/,2007,Restaurant,...,NaN,NaN,NaN,NaN,OpenTable,"OpenStreetMap, Google Maps",https://bfm2.brizodata.com/id/ef5cd4a86eec2763,Single Location,17705,96.28
1,b6f5c6b5b532b87e,58920,open,Arrowhead Grill,NaN,3/1/20,+1(623)566-2224,https://arrowheadgrill.com/,2008,Restaurant,...,NaN,NaN,NaN,NaN,OpenTable,"OpenStreetMap, Google Maps",https://bfm2.brizodata.com/id/b6f5c6b5b532b87e,Single Location,46892,98.09
2,3b6b5ed2501a5b32,12194,open,Geja's Cafe,NaN,3/1/20,+1(773)281-9101,https://www.gejascafe.com/,1965,Restaurant,...,NaN,NaN,NaN,NaN,"Tock Reservation, OpenTable",Google Maps,https://bfm2.brizodata.com/id/3b6b5ed2501a5b32,Single Location,10302,99.21
3,9b9b9ba2456ad1b6,10225,open,Ouisie's Table,NaN,3/1/20,+1(713)528-2264,https://www.ouisiestable.com/,1968,Restaurant,...,NaN,NaN,NaN,NaN,OpenTable,"Google Maps, OpenStreetMap",https://bfm2.brizodata.com/id/9b9b9ba2456ad1b6,Single Location,8691,97.49
4,3f9d376bf948dd96,64161,open,Jackson's Bistro Bar & Sushi,Jackson's Bistro,3/1/20,+1(813)277-0112,https://www.jacksonsbistro.com/,1997,"Drinking Place, Restaurant",...,NaN,NaN,NaN,NaN,"Tripleseat, OpenTable","Google Maps, OpenStreetMap",https://bfm2.brizodata.com/id/3f9d376bf948dd96,Single Location,50959,97.35


In [ ]:
# Data Overview
brizo_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9216 entries, 0 to 9215
Columns: 121 entries, Brizo_ID to Retention_Propensity
dtypes: float64(10), int64(23), object(88)
memory usage: 8.5+ MB


In [ ]:
# Select Features for the Model

selected_columns = [
    "Restaurant_ID",
    "Retention_Propensity",
    "Popmenu_Location_Segment",
    "Business_Type",
    "State/Province",
    "Density_@5mi",
    "Price_Range",
    "Median_Price",
    "Review_Score_by_Source",
    "Total_Reviews_by_Source",
    "Ratings_Followers_By_Source",
    "Technologies_Vendors",
    "Cuisines_(Continental)" 
]

brizo_selected = brizo_data[selected_columns]

# Preview the new DataFrame
print(brizo_selected.head())

   Restaurant_ID  Retention_Propensity Popmenu_Location_Segment  \
0          17705                 96.28          Single Location   
1          46892                 98.09          Single Location   
2          10302                 99.21          Single Location   
3           8691                 97.49          Single Location   
4          50959                 97.35          Single Location   

                Business_Type State/Province  Density_@5mi Price_Range  \
0                  Restaurant        Florida           324        $$$$   
1                  Restaurant        Arizona           771        $$$$   
2                  Restaurant       Illinois          6367        $$$$   
3                  Restaurant          Texas          4398          $$   
4  Drinking Place, Restaurant        Florida          1712          $$   

   Median_Price                             Review_Score_by_Source  \
0            70  Google: 4.5, Yelp: 4.3, OpenTable: 4.7, AAA To...   
1           

In [ ]:
# Copy data
lead_data = brizo_selected.copy()
lead_data.head(5)

# Normalize column names
lead_data.columns = [re.sub(r"\s+", " ", c).strip() for c in lead_data.columns]
lead_data.head(5)

,Restaurant_ID,Retention_Propensity,Popmenu_Location_Segment,Business_Type,State/Province,Density_@5mi,Price_Range,Median_Price,Review_Score_by_Source,Total_Reviews_by_Source,Ratings_Followers_By_Source,Technologies_Vendors,Cuisines_(Continental)
0,17705,96.28,Single Location,Restaurant,Florida,324,$$$$,70,"Google: 4.5, Yelp: 4.3, OpenTable: 4.7, AAA To...","Google: 2720, TripAdvisor: 2583, Yelp: 1512, O...",Facebook: 32000,"Revel Systems, Popmenu, Google Maps, Google An...",Asian
1,46892,98.09,Single Location,Restaurant,Arizona,771,$$$$,0,"Google: 4.7, TripAdvisor: 4.5, Yelp: 4.2, Open...","Google: 4224, TripAdvisor: 667, Yelp: 1650, Op...","Instagram: 1896, Facebook: 8443","OpenStreetMap, Authorize.Net, Popmenu, Google ...",North American
2,10302,99.21,Single Location,Restaurant,Illinois,6367,$$$$,40,"Google: 4.6, Gayot: 3.0, TripAdvisor: 4.0, Yel...","Google: 1854, TripAdvisor: 439, Yelp: 1676, Gr...","Instagram: 16983, Facebook: 14000","Twitter/X, Shopify, Tock Online Ordering, Tock...",European
3,8691,97.49,Single Location,Restaurant,Texas,4398,$$,48,"Google: 4.4, Gayot: 3.5, DoorDash: 5.0, The In...","Google: 1658, DoorDash: 18, TripAdvisor: 338, ...","Instagram: 2750, Facebook: 6500","Restaurant Guru, Google Analytics, Google Maps...",North American
4,50959,97.35,Single Location,"Drinking Place, Restaurant",Florida,1712,$$,32,"Google: 4.3, DoorDash: 4.4, TripAdvisor: 4.0, ...","Google: 6427, DoorDash: 21, TripAdvisor: 1884,...","Instagram: 8357, Facebook: 25341","Postmates, Facebook Pixel, OpenTable, ADP Work...",Asian


In [ ]:
# Transform Data

#Encode States and Price

# Fit and transform the State/Province column
lead_data["State/Province_Encoded"] = le.fit_transform(lead_data["State/Province"])
lead_data["Price_Range_Encoded"] = le.fit_transform(lead_data["Price_Range"])

print(lead_data.describe())
lead_data.head(5)

       Restaurant_ID  Retention_Propensity  Density_@5mi  Median_Price  \
count    9216.000000           8948.000000   9216.000000   9216.000000   
mean    24885.178060             94.442526   1239.060438     17.812500   
std     25173.094835             11.256536   2491.219408     15.863977   
min         5.000000              9.270000      1.000000      0.000000   
25%      9071.000000             96.100000    201.000000      0.000000   
50%     14734.500000             98.180000    540.000000     20.000000   
75%     24668.500000             98.830000   1307.000000     28.000000   
max    100762.000000             99.680000  26524.000000    320.000000   

       State/Province_Encoded  Price_Range_Encoded  
count             9216.000000          9216.000000  
mean                21.622938             1.278971  
std                 15.564762             1.049081  
min                  0.000000             0.000000  
25%                  9.000000             0.000000  
50%            

,Restaurant_ID,Retention_Propensity,Popmenu_Location_Segment,Business_Type,State/Province,Density_@5mi,Price_Range,Median_Price,Review_Score_by_Source,Total_Reviews_by_Source,Ratings_Followers_By_Source,Technologies_Vendors,Cuisines_(Continental),State/Province_Encoded,Price_Range_Encoded
0,17705,96.28,Single Location,Restaurant,Florida,324,$$$$,70,"Google: 4.5, Yelp: 4.3, OpenTable: 4.7, AAA To...","Google: 2720, TripAdvisor: 2583, Yelp: 1512, O...",Facebook: 32000,"Revel Systems, Popmenu, Google Maps, Google An...",Asian,9,3
1,46892,98.09,Single Location,Restaurant,Arizona,771,$$$$,0,"Google: 4.7, TripAdvisor: 4.5, Yelp: 4.2, Open...","Google: 4224, TripAdvisor: 667, Yelp: 1650, Op...","Instagram: 1896, Facebook: 8443","OpenStreetMap, Authorize.Net, Popmenu, Google ...",North American,2,3
2,10302,99.21,Single Location,Restaurant,Illinois,6367,$$$$,40,"Google: 4.6, Gayot: 3.0, TripAdvisor: 4.0, Yel...","Google: 1854, TripAdvisor: 439, Yelp: 1676, Gr...","Instagram: 16983, Facebook: 14000","Twitter/X, Shopify, Tock Online Ordering, Tock...",European,13,3
3,8691,97.49,Single Location,Restaurant,Texas,4398,$$,48,"Google: 4.4, Gayot: 3.5, DoorDash: 5.0, The In...","Google: 1658, DoorDash: 18, TripAdvisor: 338, ...","Instagram: 2750, Facebook: 6500","Restaurant Guru, Google Analytics, Google Maps...",North American,44,1
4,50959,97.35,Single Location,"Drinking Place, Restaurant",Florida,1712,$$,32,"Google: 4.3, DoorDash: 4.4, TripAdvisor: 4.0, ...","Google: 6427, DoorDash: 21, TripAdvisor: 1884,...","Instagram: 8357, Facebook: 25341","Postmates, Facebook Pixel, OpenTable, ADP Work...",Asian,9,1


In [ ]:
# Split Instagram/Facebook followers into separate columns
lead_data["Instagram_Followers"] = lead_data["Ratings_Followers_By_Source"].str.extract(r"Instagram:\s*(\d+)")
lead_data["Facebook_Followers"] = lead_data["Ratings_Followers_By_Source"].str.extract(r"Facebook:\s*(\d+)")

lead_data["Instagram_Followers"] = pd.to_numeric(lead_data["Instagram_Followers"], errors="coerce")
lead_data["Facebook_Followers"] = pd.to_numeric(lead_data["Facebook_Followers"], errors="coerce")

lead_data.head(5)

,Restaurant_ID,Retention_Propensity,Popmenu_Location_Segment,Business_Type,State/Province,Density_@5mi,Price_Range,Median_Price,Review_Score_by_Source,Total_Reviews_by_Source,Ratings_Followers_By_Source,Technologies_Vendors,Cuisines_(Continental),State/Province_Encoded,Price_Range_Encoded,Instagram_Followers,Facebook_Followers
0,17705,96.28,Single Location,Restaurant,Florida,324,$$$$,70,"Google: 4.5, Yelp: 4.3, OpenTable: 4.7, AAA To...","Google: 2720, TripAdvisor: 2583, Yelp: 1512, O...",Facebook: 32000,"Revel Systems, Popmenu, Google Maps, Google An...",Asian,9,3,NaN,32000.0
1,46892,98.09,Single Location,Restaurant,Arizona,771,$$$$,0,"Google: 4.7, TripAdvisor: 4.5, Yelp: 4.2, Open...","Google: 4224, TripAdvisor: 667, Yelp: 1650, Op...","Instagram: 1896, Facebook: 8443","OpenStreetMap, Authorize.Net, Popmenu, Google ...",North American,2,3,1896.0,8443.0
2,10302,99.21,Single Location,Restaurant,Illinois,6367,$$$$,40,"Google: 4.6, Gayot: 3.0, TripAdvisor: 4.0, Yel...","Google: 1854, TripAdvisor: 439, Yelp: 1676, Gr...","Instagram: 16983, Facebook: 14000","Twitter/X, Shopify, Tock Online Ordering, Tock...",European,13,3,16983.0,14000.0
3,8691,97.49,Single Location,Restaurant,Texas,4398,$$,48,"Google: 4.4, Gayot: 3.5, DoorDash: 5.0, The In...","Google: 1658, DoorDash: 18, TripAdvisor: 338, ...","Instagram: 2750, Facebook: 6500","Restaurant Guru, Google Analytics, Google Maps...",North American,44,1,2750.0,6500.0
4,50959,97.35,Single Location,"Drinking Place, Restaurant",Florida,1712,$$,32,"Google: 4.3, DoorDash: 4.4, TripAdvisor: 4.0, ...","Google: 6427, DoorDash: 21, TripAdvisor: 1884,...","Instagram: 8357, Facebook: 25341","Postmates, Facebook Pixel, OpenTable, ADP Work...",Asian,9,1,8357.0,25341.0


In [ ]:
# Split review score

# Apply parsing
review_dicts = lead_data["Review_Score_by_Source"].apply(parse_selected_scores)

# Expand into separate columns
review_scores = pd.json_normalize(review_dicts)

# Add suffix to keep column names clear
review_scores = review_scores.add_suffix("_Review_Score")

# Merge back to original dataframe
lead_data = pd.concat([lead_data, review_scores], axis=1)

lead_data.head(5)

,Restaurant_ID,Retention_Propensity,Popmenu_Location_Segment,Business_Type,State/Province,Density_@5mi,Price_Range,Median_Price,Review_Score_by_Source,Total_Reviews_by_Source,...,Cuisines_(Continental),State/Province_Encoded,Price_Range_Encoded,Instagram_Followers,Facebook_Followers,Google_Review_Score,OpenTable_Review_Score,TripAdvisor_Review_Score,Facebook_Review_Score,DoorDash_Review_Score
0,17705,96.28,Single Location,Restaurant,Florida,324,$$$$,70,"Google: 4.5, Yelp: 4.3, OpenTable: 4.7, AAA To...","Google: 2720, TripAdvisor: 2583, Yelp: 1512, O...",...,Asian,9,3,NaN,32000.0,4.5,4.7,NaN,NaN,NaN
1,46892,98.09,Single Location,Restaurant,Arizona,771,$$$$,0,"Google: 4.7, TripAdvisor: 4.5, Yelp: 4.2, Open...","Google: 4224, TripAdvisor: 667, Yelp: 1650, Op...",...,North American,2,3,1896.0,8443.0,4.7,4.8,4.5,4.7,NaN
2,10302,99.21,Single Location,Restaurant,Illinois,6367,$$$$,40,"Google: 4.6, Gayot: 3.0, TripAdvisor: 4.0, Yel...","Google: 1854, TripAdvisor: 439, Yelp: 1676, Gr...",...,European,13,3,16983.0,14000.0,4.6,4.9,4.0,NaN,NaN
3,8691,97.49,Single Location,Restaurant,Texas,4398,$$,48,"Google: 4.4, Gayot: 3.5, DoorDash: 5.0, The In...","Google: 1658, DoorDash: 18, TripAdvisor: 338, ...",...,North American,44,1,2750.0,6500.0,4.4,4.8,4.0,NaN,5.0
4,50959,97.35,Single Location,"Drinking Place, Restaurant",Florida,1712,$$,32,"Google: 4.3, DoorDash: 4.4, TripAdvisor: 4.0, ...","Google: 6427, DoorDash: 21, TripAdvisor: 1884,...",...,Asian,9,1,8357.0,25341.0,4.3,4.5,4.0,4.4,4.4


In [ ]:

# Split reviews

# Apply parsing
total_review_dicts = lead_data["Total_Reviews_by_Source"].apply(parse_selected_scores)

# Expand into separate columns
reviews = pd.json_normalize(total_review_dicts)

# Add suffix to keep column names clear
reviews = reviews.add_suffix("_Total_Reviews")

# Merge back to original dataframe
lead_data = pd.concat([lead_data, reviews], axis=1)
lead_data.head(5)

,Restaurant_ID,Retention_Propensity,Popmenu_Location_Segment,Business_Type,State/Province,Density_@5mi,Price_Range,Median_Price,Review_Score_by_Source,Total_Reviews_by_Source,...,Google_Review_Score,OpenTable_Review_Score,TripAdvisor_Review_Score,Facebook_Review_Score,DoorDash_Review_Score,Google_Total_Reviews,TripAdvisor_Total_Reviews,OpenTable_Total_Reviews,Facebook_Total_Reviews,DoorDash_Total_Reviews
0,17705,96.28,Single Location,Restaurant,Florida,324,$$$$,70,"Google: 4.5, Yelp: 4.3, OpenTable: 4.7, AAA To...","Google: 2720, TripAdvisor: 2583, Yelp: 1512, O...",...,4.5,4.7,NaN,NaN,NaN,2720.0,2583.0,8277.0,487.0,NaN
1,46892,98.09,Single Location,Restaurant,Arizona,771,$$$$,0,"Google: 4.7, TripAdvisor: 4.5, Yelp: 4.2, Open...","Google: 4224, TripAdvisor: 667, Yelp: 1650, Op...",...,4.7,4.8,4.5,4.7,NaN,4224.0,667.0,9555.0,456.0,NaN
2,10302,99.21,Single Location,Restaurant,Illinois,6367,$$$$,40,"Google: 4.6, Gayot: 3.0, TripAdvisor: 4.0, Yel...","Google: 1854, TripAdvisor: 439, Yelp: 1676, Gr...",...,4.6,4.9,4.0,NaN,NaN,1854.0,439.0,5730.0,854.0,NaN
3,8691,97.49,Single Location,Restaurant,Texas,4398,$$,48,"Google: 4.4, Gayot: 3.5, DoorDash: 5.0, The In...","Google: 1658, DoorDash: 18, TripAdvisor: 338, ...",...,4.4,4.8,4.0,NaN,5.0,1658.0,338.0,4902.0,419.0,18.0
4,50959,97.35,Single Location,"Drinking Place, Restaurant",Florida,1712,$$,32,"Google: 4.3, DoorDash: 4.4, TripAdvisor: 4.0, ...","Google: 6427, DoorDash: 21, TripAdvisor: 1884,...",...,4.3,4.5,4.0,4.4,4.4,6427.0,1884.0,5186.0,2290.0,21.0


In [ ]:
preferred_vendors = ["Google", "Facebook", "Toast", "Instagram", "DoorDash", "OpenTable", "Square", "Stripe", "SpotOn", "Aloha", "Heartland", "Clover", "Symphony", "Deliverect"]

# --- Create boolean columns only for preferred vendors ---
for vendor in preferred_vendors:
    col_name = f"Tech_Preferred_{vendor.replace(' ', '_')}"  # make column name safe
    lead_data[col_name] = lead_data["Technologies_Vendors"].str.contains(vendor, case=False, na=False).astype(int)

lead_data.head(5)

,Restaurant_ID,Retention_Propensity,Popmenu_Location_Segment,Business_Type,State/Province,Density_@5mi,Price_Range,Median_Price,Review_Score_by_Source,Total_Reviews_by_Source,...,Tech_Preferred_DoorDash,Tech_Preferred_OpenTable,Tech_Preferred_Square,Tech_Preferred_Stripe,Tech_Preferred_SpotOn,Tech_Preferred_Aloha,Tech_Preferred_Heartland,Tech_Preferred_Clover,Tech_Preferred_Symphony,Tech_Preferred_Deliverect
0,17705,96.28,Single Location,Restaurant,Florida,324,$$$$,70,"Google: 4.5, Yelp: 4.3, OpenTable: 4.7, AAA To...","Google: 2720, TripAdvisor: 2583, Yelp: 1512, O...",...,0,1,0,0,0,0,0,0,0,0
1,46892,98.09,Single Location,Restaurant,Arizona,771,$$$$,0,"Google: 4.7, TripAdvisor: 4.5, Yelp: 4.2, Open...","Google: 4224, TripAdvisor: 667, Yelp: 1650, Op...",...,0,1,0,0,0,0,0,0,0,0
2,10302,99.21,Single Location,Restaurant,Illinois,6367,$$$$,40,"Google: 4.6, Gayot: 3.0, TripAdvisor: 4.0, Yel...","Google: 1854, TripAdvisor: 439, Yelp: 1676, Gr...",...,0,1,1,0,0,0,0,0,0,0
3,8691,97.49,Single Location,Restaurant,Texas,4398,$$,48,"Google: 4.4, Gayot: 3.5, DoorDash: 5.0, The In...","Google: 1658, DoorDash: 18, TripAdvisor: 338, ...",...,1,1,0,0,0,0,0,0,0,0
4,50959,97.35,Single Location,"Drinking Place, Restaurant",Florida,1712,$$,32,"Google: 4.3, DoorDash: 4.4, TripAdvisor: 4.0, ...","Google: 6427, DoorDash: 21, TripAdvisor: 1884,...",...,1,1,0,0,0,0,0,0,0,0


In [ ]:
competitor_vendors = ["ezCater","Chownow","Spothopper","Menufy","Olo","Bentobox","MailChimp","HungerRush","Incentivio","Owner","Klaviyo","Chowly"]

# --- Create boolean columns only for preferred vendors ---
for vendor in competitor_vendors:
    col_name = f"Tech_Competitor_{vendor.replace(' ', '_')}"  # make column name safe
    lead_data[col_name] = lead_data["Technologies_Vendors"].str.contains(vendor, case=False, na=False).astype(int)

lead_data.head(5)

,Restaurant_ID,Retention_Propensity,Popmenu_Location_Segment,Business_Type,State/Province,Density_@5mi,Price_Range,Median_Price,Review_Score_by_Source,Total_Reviews_by_Source,...,Tech_Competitor_Spothopper,Tech_Competitor_Menufy,Tech_Competitor_Olo,Tech_Competitor_Bentobox,Tech_Competitor_MailChimp,Tech_Competitor_HungerRush,Tech_Competitor_Incentivio,Tech_Competitor_Owner,Tech_Competitor_Klaviyo,Tech_Competitor_Chowly
0,17705,96.28,Single Location,Restaurant,Florida,324,$$$$,70,"Google: 4.5, Yelp: 4.3, OpenTable: 4.7, AAA To...","Google: 2720, TripAdvisor: 2583, Yelp: 1512, O...",...,0,0,0,0,0,0,0,0,0,0
1,46892,98.09,Single Location,Restaurant,Arizona,771,$$$$,0,"Google: 4.7, TripAdvisor: 4.5, Yelp: 4.2, Open...","Google: 4224, TripAdvisor: 667, Yelp: 1650, Op...",...,0,0,0,0,0,0,0,0,0,0
2,10302,99.21,Single Location,Restaurant,Illinois,6367,$$$$,40,"Google: 4.6, Gayot: 3.0, TripAdvisor: 4.0, Yel...","Google: 1854, TripAdvisor: 439, Yelp: 1676, Gr...",...,0,0,0,0,0,0,0,0,0,0
3,8691,97.49,Single Location,Restaurant,Texas,4398,$$,48,"Google: 4.4, Gayot: 3.5, DoorDash: 5.0, The In...","Google: 1658, DoorDash: 18, TripAdvisor: 338, ...",...,0,1,0,0,0,0,0,0,0,0
4,50959,97.35,Single Location,"Drinking Place, Restaurant",Florida,1712,$$,32,"Google: 4.3, DoorDash: 4.4, TripAdvisor: 4.0, ...","Google: 6427, DoorDash: 21, TripAdvisor: 1884,...",...,0,0,0,0,0,0,0,0,0,0


In [ ]:
overlapping_vendors = ["Uber Eats","Grubhub","Seamless","Postmates","Caviar","Paytronix"]

# --- Create boolean columns only for preferred vendors ---
for vendor in overlapping_vendors:
    col_name = f"Tech_Overlapping_{vendor.replace(' ', '_')}"  # make column name safe
    lead_data[col_name] = lead_data["Technologies_Vendors"].str.contains(vendor, case=False, na=False).astype(int)

lead_data.head(5)

,Restaurant_ID,Retention_Propensity,Popmenu_Location_Segment,Business_Type,State/Province,Density_@5mi,Price_Range,Median_Price,Review_Score_by_Source,Total_Reviews_by_Source,...,Tech_Competitor_Incentivio,Tech_Competitor_Owner,Tech_Competitor_Klaviyo,Tech_Competitor_Chowly,Tech_Overlapping_Uber_Eats,Tech_Overlapping_Grubhub,Tech_Overlapping_Seamless,Tech_Overlapping_Postmates,Tech_Overlapping_Caviar,Tech_Overlapping_Paytronix
0,17705,96.28,Single Location,Restaurant,Florida,324,$$$$,70,"Google: 4.5, Yelp: 4.3, OpenTable: 4.7, AAA To...","Google: 2720, TripAdvisor: 2583, Yelp: 1512, O...",...,0,0,0,0,0,0,0,0,0,0
1,46892,98.09,Single Location,Restaurant,Arizona,771,$$$$,0,"Google: 4.7, TripAdvisor: 4.5, Yelp: 4.2, Open...","Google: 4224, TripAdvisor: 667, Yelp: 1650, Op...",...,0,0,0,0,0,0,0,0,0,0
2,10302,99.21,Single Location,Restaurant,Illinois,6367,$$$$,40,"Google: 4.6, Gayot: 3.0, TripAdvisor: 4.0, Yel...","Google: 1854, TripAdvisor: 439, Yelp: 1676, Gr...",...,0,0,0,0,0,0,0,0,0,0
3,8691,97.49,Single Location,Restaurant,Texas,4398,$$,48,"Google: 4.4, Gayot: 3.5, DoorDash: 5.0, The In...","Google: 1658, DoorDash: 18, TripAdvisor: 338, ...",...,0,0,0,0,1,1,1,1,1,0
4,50959,97.35,Single Location,"Drinking Place, Restaurant",Florida,1712,$$,32,"Google: 4.3, DoorDash: 4.4, TripAdvisor: 4.0, ...","Google: 6427, DoorDash: 21, TripAdvisor: 1884,...",...,0,0,0,0,1,0,0,1,0,0


In [ ]:
cusine_type = ["North American","European","Latin American","Asian"]

# --- Create boolean columns only for preferred vendors ---
for cusine in cusine_type:
    col_name = f"Cusine_{cusine.replace(' ', '_')}"  # make column name safe
    lead_data[col_name] = lead_data["Cuisines_(Continental)"].str.contains(cusine, case=False, na=False).astype(int)

lead_data.head(5)

,Restaurant_ID,Retention_Propensity,Popmenu_Location_Segment,Business_Type,State/Province,Density_@5mi,Price_Range,Median_Price,Review_Score_by_Source,Total_Reviews_by_Source,...,Tech_Overlapping_Uber_Eats,Tech_Overlapping_Grubhub,Tech_Overlapping_Seamless,Tech_Overlapping_Postmates,Tech_Overlapping_Caviar,Tech_Overlapping_Paytronix,Cusine_North_American,Cusine_European,Cusine_Latin_American,Cusine_Asian
0,17705,96.28,Single Location,Restaurant,Florida,324,$$$$,70,"Google: 4.5, Yelp: 4.3, OpenTable: 4.7, AAA To...","Google: 2720, TripAdvisor: 2583, Yelp: 1512, O...",...,0,0,0,0,0,0,0,0,0,1
1,46892,98.09,Single Location,Restaurant,Arizona,771,$$$$,0,"Google: 4.7, TripAdvisor: 4.5, Yelp: 4.2, Open...","Google: 4224, TripAdvisor: 667, Yelp: 1650, Op...",...,0,0,0,0,0,0,1,0,0,0
2,10302,99.21,Single Location,Restaurant,Illinois,6367,$$$$,40,"Google: 4.6, Gayot: 3.0, TripAdvisor: 4.0, Yel...","Google: 1854, TripAdvisor: 439, Yelp: 1676, Gr...",...,0,0,0,0,0,0,0,1,0,0
3,8691,97.49,Single Location,Restaurant,Texas,4398,$$,48,"Google: 4.4, Gayot: 3.5, DoorDash: 5.0, The In...","Google: 1658, DoorDash: 18, TripAdvisor: 338, ...",...,1,1,1,1,1,0,1,0,0,0
4,50959,97.35,Single Location,"Drinking Place, Restaurant",Florida,1712,$$,32,"Google: 4.3, DoorDash: 4.4, TripAdvisor: 4.0, ...","Google: 6427, DoorDash: 21, TripAdvisor: 1884,...",...,1,0,0,1,0,0,0,0,0,1


In [ ]:
business_type = ["Restaurant","Drinking Place","Quick Service","Cafe"]

# --- Create boolean columns only for preferred vendors ---
for business in business_type:
    col_name = f"Business_Type_{business.replace(' ', '_')}"  # make column name safe
    lead_data[col_name] = lead_data["Business_Type"].str.contains(business, case=False, na=False).astype(int)

lead_data.head(5)

,Restaurant_ID,Retention_Propensity,Popmenu_Location_Segment,Business_Type,State/Province,Density_@5mi,Price_Range,Median_Price,Review_Score_by_Source,Total_Reviews_by_Source,...,Tech_Overlapping_Caviar,Tech_Overlapping_Paytronix,Cusine_North_American,Cusine_European,Cusine_Latin_American,Cusine_Asian,Business_Type_Restaurant,Business_Type_Drinking_Place,Business_Type_Quick_Service,Business_Type_Cafe
0,17705,96.28,Single Location,Restaurant,Florida,324,$$$$,70,"Google: 4.5, Yelp: 4.3, OpenTable: 4.7, AAA To...","Google: 2720, TripAdvisor: 2583, Yelp: 1512, O...",...,0,0,0,0,0,1,1,0,0,0
1,46892,98.09,Single Location,Restaurant,Arizona,771,$$$$,0,"Google: 4.7, TripAdvisor: 4.5, Yelp: 4.2, Open...","Google: 4224, TripAdvisor: 667, Yelp: 1650, Op...",...,0,0,1,0,0,0,1,0,0,0
2,10302,99.21,Single Location,Restaurant,Illinois,6367,$$$$,40,"Google: 4.6, Gayot: 3.0, TripAdvisor: 4.0, Yel...","Google: 1854, TripAdvisor: 439, Yelp: 1676, Gr...",...,0,0,0,1,0,0,1,0,0,0
3,8691,97.49,Single Location,Restaurant,Texas,4398,$$,48,"Google: 4.4, Gayot: 3.5, DoorDash: 5.0, The In...","Google: 1658, DoorDash: 18, TripAdvisor: 338, ...",...,1,0,1,0,0,0,1,0,0,0
4,50959,97.35,Single Location,"Drinking Place, Restaurant",Florida,1712,$$,32,"Google: 4.3, DoorDash: 4.4, TripAdvisor: 4.0, ...","Google: 6427, DoorDash: 21, TripAdvisor: 1884,...",...,0,0,0,0,0,1,1,1,0,0


In [ ]:
# Drop columns if they exist in the DataFrame
columns_to_drop = [
    "Review_Score_by_Source",
    "Total_Reviews_by_Source",
    "Ratings_Followers_By_Source",
    "Technologies_Vendors",
    "Business_Type",
    "Price_Range"
]

lead_data = lead_data.drop(columns=[col for col in columns_to_drop if col in lead_data.columns], errors='ignore')

# Display the first 5 rows
lead_data.head(5)

,Restaurant_ID,Retention_Propensity,Popmenu_Location_Segment,State/Province,Density_@5mi,Median_Price,Cuisines_(Continental),State/Province_Encoded,Price_Range_Encoded,Instagram_Followers,...,Tech_Overlapping_Caviar,Tech_Overlapping_Paytronix,Cusine_North_American,Cusine_European,Cusine_Latin_American,Cusine_Asian,Business_Type_Restaurant,Business_Type_Drinking_Place,Business_Type_Quick_Service,Business_Type_Cafe
0,17705,96.28,Single Location,Florida,324,70,Asian,9,3,NaN,...,0,0,0,0,0,1,1,0,0,0
1,46892,98.09,Single Location,Arizona,771,0,North American,2,3,1896.0,...,0,0,1,0,0,0,1,0,0,0
2,10302,99.21,Single Location,Illinois,6367,40,European,13,3,16983.0,...,0,0,0,1,0,0,1,0,0,0
3,8691,97.49,Single Location,Texas,4398,48,North American,44,1,2750.0,...,1,0,1,0,0,0,1,0,0,0
4,50959,97.35,Single Location,Florida,1712,32,Asian,9,1,8357.0,...,0,0,0,0,0,1,1,1,0,0


In [ ]:
lead_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9216 entries, 0 to 9215
Data columns (total 61 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Restaurant_ID                 9216 non-null   int64  
 1   Retention_Propensity          8948 non-null   float64
 2   Popmenu_Location_Segment      9216 non-null   object 
 3   State/Province                9216 non-null   object 
 4   Density_@5mi                  9216 non-null   int64  
 5   Median_Price                  9216 non-null   int64  
 6   Cuisines_(Continental)        8424 non-null   object 
 7   State/Province_Encoded        9216 non-null   int64  
 8   Price_Range_Encoded           9216 non-null   int64  
 9   Instagram_Followers           4934 non-null   float64
 10  Facebook_Followers            6066 non-null   float64
 11  Google_Review_Score           9054 non-null   float64
 12  OpenTable_Review_Score        1418 non-null   float64
 13  Tri

In [ ]:
# convert to string

cols_to_convert = ["Restaurant_ID"]
lead_data[cols_to_convert] = lead_data[cols_to_convert].astype(str)

# Remove rows where Retention_Propensity is NaN or empty
lead_data = lead_data.dropna(subset=['Retention_Propensity'])


lead_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8948 entries, 0 to 9215
Data columns (total 61 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Restaurant_ID                 8948 non-null   object 
 1   Retention_Propensity          8948 non-null   float64
 2   Popmenu_Location_Segment      8948 non-null   object 
 3   State/Province                8948 non-null   object 
 4   Density_@5mi                  8948 non-null   int64  
 5   Median_Price                  8948 non-null   int64  
 6   Cuisines_(Continental)        8190 non-null   object 
 7   State/Province_Encoded        8948 non-null   int64  
 8   Price_Range_Encoded           8948 non-null   int64  
 9   Instagram_Followers           4838 non-null   float64
 10  Facebook_Followers            5929 non-null   float64
 11  Google_Review_Score           8794 non-null   float64
 12  OpenTable_Review_Score        1386 non-null   float64
 13  TripAdvi

In [ ]:
# Check for Duplicates
lead_data.duplicated().sum()    

np.int64(0)

In [ ]:
# Check for Unique Values
lead_data.nunique()

Restaurant_ID                   6254
Retention_Propensity            1486
Popmenu_Location_Segment           2
State/Province                    52
Density_@5mi                    2842
                                ... 
Cusine_Asian                       2
Business_Type_Restaurant           2
Business_Type_Drinking_Place       2
Business_Type_Quick_Service        2
Business_Type_Cafe                 2
Length: 61, dtype: int64

In [ ]:
# Check for Summary Statistics
lead_data.describe()

,Retention_Propensity,Density_@5mi,Median_Price,State/Province_Encoded,Price_Range_Encoded,Instagram_Followers,Facebook_Followers,Google_Review_Score,OpenTable_Review_Score,TripAdvisor_Review_Score,...,Tech_Overlapping_Caviar,Tech_Overlapping_Paytronix,Cusine_North_American,Cusine_European,Cusine_Latin_American,Cusine_Asian,Business_Type_Restaurant,Business_Type_Drinking_Place,Business_Type_Quick_Service,Business_Type_Cafe
count,8948.000000,8948.000000,8948.000000,8948.000000,8948.000000,4.838000e+03,5929.000000,8794.000000,1386.000000,2534.000000,...,8948.000000,8948.000000,8948.000000,8948.000000,8948.000000,8948.000000,8948.000000,8948.000000,8948.000000,8948.000000
mean,94.442526,1239.210773,17.816272,21.688869,1.287774,1.561029e+04,3999.054309,4.381226,4.491342,3.576953,...,0.246200,0.020116,0.516987,0.177805,0.173223,0.078118,0.885673,0.332588,0.214685,0.043473
std,11.256536,2505.699054,15.961624,15.581755,1.049250,8.713049e+04,6357.836274,0.279882,0.347966,1.318129,...,0.430821,0.140406,0.499739,0.382370,0.378461,0.268372,0.318226,0.471167,0.410626,0.203932
min,9.270000,1.000000,0.000000,0.000000,0.000000,2.000000e+00,1.000000,1.000000,1.600000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,96.100000,200.000000,0.000000,9.000000,0.000000,1.138250e+03,999.000000,4.200000,4.400000,3.500000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
50%,98.180000,537.000000,20.000000,19.000000,1.000000,3.131000e+03,2341.000000,4.400000,4.600000,4.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,98.830000,1298.000000,28.000000,35.000000,2.000000,8.678750e+03,4854.000000,4.500000,4.700000,4.500000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
max,99.680000,26524.000000,320.000000,51.000000,4.000000,2.043234e+06,196182.000000,5.000000,4.900000,5.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
# define numerical & categorical columns
numeric_features = [feature for feature in lead_data.columns if lead_data[feature].dtype != 'O']
categorical_features = [feature for feature in lead_data.columns if lead_data[feature].dtype == 'O']

# print columns
print('We have {} numerical features : {}'.format(len(numeric_features), numeric_features))
print('\nWe have {} categorical features : {}'.format(len(categorical_features), categorical_features))

We have 57 numerical features : ['Retention_Propensity', 'Density_@5mi', 'Median_Price', 'State/Province_Encoded', 'Price_Range_Encoded', 'Instagram_Followers', 'Facebook_Followers', 'Google_Review_Score', 'OpenTable_Review_Score', 'TripAdvisor_Review_Score', 'Facebook_Review_Score', 'DoorDash_Review_Score', 'Google_Total_Reviews', 'TripAdvisor_Total_Reviews', 'OpenTable_Total_Reviews', 'Facebook_Total_Reviews', 'DoorDash_Total_Reviews', 'Tech_Preferred_Google', 'Tech_Preferred_Facebook', 'Tech_Preferred_Toast', 'Tech_Preferred_Instagram', 'Tech_Preferred_DoorDash', 'Tech_Preferred_OpenTable', 'Tech_Preferred_Square', 'Tech_Preferred_Stripe', 'Tech_Preferred_SpotOn', 'Tech_Preferred_Aloha', 'Tech_Preferred_Heartland', 'Tech_Preferred_Clover', 'Tech_Preferred_Symphony', 'Tech_Preferred_Deliverect', 'Tech_Competitor_ezCater', 'Tech_Competitor_Chownow', 'Tech_Competitor_Spothopper', 'Tech_Competitor_Menufy', 'Tech_Competitor_Olo', 'Tech_Competitor_Bentobox', 'Tech_Competitor_MailChimp', '

In [ ]:
# Replace all NaN or null values in all columns with 0
lead_data = lead_data.fillna(0)

# Verify that no missing values remain
print(lead_data.isna().sum().sum())  # Should print 0

lead_data.info()

0
<class 'pandas.core.frame.DataFrame'>
Index: 8948 entries, 0 to 9215
Data columns (total 61 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Restaurant_ID                 8948 non-null   object 
 1   Retention_Propensity          8948 non-null   float64
 2   Popmenu_Location_Segment      8948 non-null   object 
 3   State/Province                8948 non-null   object 
 4   Density_@5mi                  8948 non-null   int64  
 5   Median_Price                  8948 non-null   int64  
 6   Cuisines_(Continental)        8948 non-null   object 
 7   State/Province_Encoded        8948 non-null   int64  
 8   Price_Range_Encoded           8948 non-null   int64  
 9   Instagram_Followers           8948 non-null   float64
 10  Facebook_Followers            8948 non-null   float64
 11  Google_Review_Score           8948 non-null   float64
 12  OpenTable_Review_Score        8948 non-null   float64
 13  TripAd

## Data transformation summary and column descriptions

### Purpose / contract
- Input: original Brizo export CSV with columns including Restaurant_ID, Retention_Propensity, Popmenu_Location_Segment, Business_Type, State/Province, Density_@5mi, Price_Range, Median_Price, Review_Score_by_Source, Total_Reviews_by_Source, Ratings_Followers_By_Source, Technologies_Vendors, Cuisines_(Continental)
- Output: cleaned `lead_data` DataFrame prepared for modelling with encoded categorical features, extracted numeric metrics, and many vendor/cuisine/business boolean flags.
- Error modes: parsing functions return missing/empty results if source text is NaN or not in expected format.

---

### Transformations performed (step-by-step)
1. Column name normalization
   - Replaced multiple whitespace in column names and trimmed leading/trailing spaces so column names are consistent.

2. Feature selection
   - Kept a set of selected columns from the raw CSV into `brizo_selected` and then copied into `lead_data` for transformations.

3. Encoding categorical variables
   - Created `State/Province_Encoded` using a LabelEncoder instance (`le`).
   - Created `Price_Range_Encoded` using the same LabelEncoder instance (fit on the `Price_Range` column first).
   - Note: the encoded columns allow tree/linear models to consume categorical information numerically; if you prefer one-hot encoding for linear models, apply OneHotEncoder later.

4. Followers extraction
   - Parsed `Ratings_Followers_By_Source` to extract numeric follower counts:
     - `Instagram_Followers` and `Facebook_Followers` were extracted using regex and converted to numeric (coercing non-numeric to NaN).

5. Review score and review count parsing
   - Used `parse_selected_scores` to parse `Review_Score_by_Source` and `Total_Reviews_by_Source` strings (expected format: `Source: value, Source2: value, ...`).
   - Kept only chosen sources (Google, DoorDash, TripAdvisor, OpenTable, Facebook) and created separate numeric columns for each source's score and total reviews.
   - After normalization, review score columns were suffixed with `_Review_Score` and review counts with `_Total_Reviews`.

6. Technology/vendor flags
   - Created boolean (0/1) columns for many vendor names across three groups:
     - Preferred vendors: columns named `Tech_Preferred_<Vendor>`.
     - Competitor vendors: `Tech_Competitor_<Vendor>`.
     - Overlapping vendors: `Tech_Overlapping_<Vendor>`.
   - Each is generated by checking whether the `Technologies_Vendors` string contains the vendor name (case-insensitive). Missing values are treated as not containing the vendor (na=False).

7. Cuisine and business type flags
   - Created boolean columns `Cusine_<Type>` for listed continental cuisine types (e.g., `Cusine_North_American`).
   - Created `Business_Type_<Type>` booleans for common business types (Restaurant, Drinking Place, Quick Service, Cafe).

8. Column drops and final cleanup
   - Dropped raw text columns that were parsed or converted into structured features to avoid duplication: `Review_Score_by_Source`, `Total_Reviews_by_Source`, `Ratings_Followers_By_Source`, `Technologies_Vendors`, `Business_Type`, `Price_Range` (these exist as encoded columns now).
   - Converted `Restaurant_ID` to string explicitly to preserve identifiers.
   - Dropped rows where `Retention_Propensity` is missing to ensure the target is available for modelling.
   - Ran basic exploratory checks: duplicates, unique counts, info(), describe() show structure and summary statistics.

---

### Column descriptions (final / produced columns - summary)
Below is a high-level description of the important columns present in `lead_data` after transformation. Not every auto-generated boolean column is listed — vendor/cuisine/business booleans follow the naming patterns described above.

- Restaurant_ID (string): unique identifier for each restaurant (converted to string type).
- Retention_Propensity (numeric): target variable; rows without this value were dropped.
- Popmenu_Location_Segment (categorical/string): original segment grouping from source.
- State/Province (original string): original state/province (kept unless later dropped). Use `State/Province_Encoded` for numeric representation.
- State/Province_Encoded (integer): Label-encoded representation of `State/Province`.
- Density_@5mi (numeric): numeric density feature from source (kept as-is).
- Median_Price (numeric): numeric median price for the restaurant; retained.
- Price_Range_Encoded (integer): Label-encoded representation of `Price_Range`. Useful when `Price_Range` is ordinal.

- Instagram_Followers (numeric): parsed follower count from `Ratings_Followers_By_Source` (NaN when not provided).
- Facebook_Followers (numeric): parsed follower count from `Ratings_Followers_By_Source` (NaN when not provided).

- <Source>_Review_Score (float): parsed review score for each selected source (Google, DoorDash, TripAdvisor, OpenTable, Facebook) — e.g., `Google_Review_Score`.
- <Source>_Total_Reviews (numeric): parsed review count for each selected source — e.g., `Google_Total_Reviews`.

- Tech_Preferred_<Vendor> (0/1): flags indicating presence of a preferred vendor in the `Technologies_Vendors` string (e.g., `Tech_Preferred_Google`, `Tech_Preferred_Facebook`).
- Tech_Competitor_<Vendor> (0/1): flags for competitor vendors (e.g., `Tech_Competitor_ezCater`).
- Tech_Overlapping_<Vendor> (0/1): flags for overlapping vendors (delivery/ordering marketplaces, etc.).

- Cusine_<Type> (0/1): flags for continental cuisine categories (e.g., `Cusine_North_American`).
- Business_Type_<Type> (0/1): flags for business types (e.g., `Business_Type_Restaurant`).

- Any other columns created by the parsing or flagging will use the patterns above (suffixes and prefixes). Use column name patterns to locate groups of features when building pipelines (e.g., columns starting with `Tech_Preferred_`, `Cusine_`, or ending with `_Review_Score`).

---

### Notes, assumptions and next steps
- Assumptions:
  - `parse_selected_scores` expects source text as `"Source: value, Source2: value"`. If the raw text varies significantly, the parser may need to be hardened (strip non-numeric characters, support parentheses, percent signs, etc.).
  - LabelEncoder `le` was reused for both `State/Province` and `Price_Range` in the notebook as written; this means the same encoder instance is fitted twice. For production pipelines, create separate encoder instances or use a persistent mapping to avoid label collisions across columns.

- Data quality / edge cases to review:
  - Several parsed numeric columns can have NaN where the source omitted information — consider imputing or adding a `missing` indicator.
  - Count outliers: follower counts and total reviews may require log-scaling for models sensitive to skew.
  - Vendors with similar names (spaces, punctuation) may create false negatives; normalize vendor strings (remove punctuation) if that becomes an issue.

- Next steps you might take in this notebook:
  1. Add explicit imputation rules (SimpleImputer) and scaling (StandardScaler) in a pipeline.
  2. Export or persist encoders/mappings so training and inference use identical transforms.
  3. Create a column mapping table (CSV) listing all auto-generated columns for documentation and model feature selection.
  4. Add automated checks (assertions) for expected column presence and types after the transform.

If you'd like, I can also (a) add the separate encoder instances and a short demonstration of a sklearn ColumnTransformer for numeric/categorical pipelines, or (b) write a small CSV that lists every column with a single-line description for easier hand-off.